--- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..
...##.
.####.
....#.
s###..

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?


In [1]:
def inputWrangler(filepath):
    '''
    Read the input and returns a list of lists with [direction and amount].
    '''
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
    return [i.strip().split() for i in lines]

def day09a(filepath='input09.txt'):
    '''
    
    '''
    
    instructions = inputWrangler(filepath)
    #position head and tail
    posH, posT = [0,0], [0,0]
    #keep a list of tail positions
    tailVisited = [[0,0]]
    for instruction in instructions:
        direction, amount = instruction[0], int(instruction[1])
        #print(direction, amount, posH, posT, len(tailVisited))
        #determine the direction
        for i in range(amount):
            prevH = [i for i in posH]
            if direction == 'R':
                posH[0] += 1
            elif direction == 'L':
                posH[0] -= 1
            elif direction == 'U':
                posH[1] += 1
            elif direction == 'D':
                posH[1] -= 1
            if (posH[0]-posT[0])**2 + (posH[1]-posT[1])**2 > 2:
                    posT = prevH
                    tailVisited.append(posT)
    

    ans = len(set((i[0], i[1]) for i in tailVisited))
    print(f"The tail visited {ans} positions.")
        
    #if you want to print the board
    #board = [[0 for i in range(1000)] for i in range(1000)]
    #for pos in tailVisited:
    #    #print(pos)
    #    board[pos[1]][pos[0]] = 1
    #for row in board:
    #   #print(row)
day09a(filepath='test09.txt')    
day09a()

The tail visited 13 positions.
The tail visited 6367 positions.


In [47]:
def day09b(filepath='input09.txt', Nknots=10):
    '''
    
    '''
    
    instructions = inputWrangler(filepath)
    #position head and tail
    instructions = [[instruction[0], int(instruction[1])] for instruction in instructions]
    directionDict = {'U': [0,1], "D": [0,-1], "L": [-1, 0], "R": [1,0]}
    knots = [[0,0] for i in range(Nknots)]
    knotMoves = {(2,0): (1,0), (-2, 0): (-1, 0), (0, 2): (0, 1), (0, -2): (0, -1),
                    (1,2): (1,1), (2,1): (1,1), (-1, 2): (-1, 1), (-2, 1): (-1, 1),
                    (-1,-2): (-1,-1), (-2,-1): (-1,-1), (1,-2): (1, -1), (2, -1): (1, -1),
                    (2,2): (1,1), (2, -2):(1,-1), (-2,-2):(-1,-1), (-2,2):(-1,1)}
    tailVisited = set()
    
    for direction, amount in instructions:
        for i in range(amount):
            #iterate through the knots with knot number
            moveX, moveY = directionDict[direction]
            knots[0] = (knots[0][0] + moveX, knots[0][1] + moveY)
            
            for j in range(1, Nknots):
                knot1, knot2 = knots[j-1], knots[j]
                knots[j-1] = knot1
                dx, dy = (knot1[0]-knot2[0]) , (knot1[1]-knot2[1])
                delta = dx**2 + dy**2
                if delta <= 2:
                    pass
                else:
                    knot2 = (knot2[0]+knotMoves[(dx,dy)][0], knot2[1]+knotMoves[(dx,dy)][1])
                    knots[j] = knot2
                if j == Nknots-1:
                    tailVisited.add(tuple(knot2))
    print(len(tailVisited))                
    #print((tailVisited))   

#day09b(filepath='test09.txt', Nknots=2)
day09b(filepath='test09_1.txt')    
day09b()

36
2536
